In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
def generate_dataset(cascade_path, filepath, filename, n = 200):
    face_cascade = cv2.CascadeClassifier(cascade_path)
    
    def crop_image(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    id = 1
    img_id = 0
    while True:
        ret, frame = cap.read()
        
        if crop_image(frame) is not None:
            img_id += 1
            face = cv2.resize(crop_image(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"{filepath}\\{filename}." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            
            cv2.imshow("Captured Face", face)
            if cv2.waitKey(1) == 13 or int(img_id) == n:
                break
                
    cap.release()
    cv2.destroyAllWindows()
    print('Collecting images completed...')

In [4]:
cascade_path = r'C:\Users\user\Downloads\4.1\haarcascade_frontalface_default.xml'
happy = r"C:\Users\user\Downloads\4.1\MOOD\me\happy"
sad = r"C:\Users\user\Downloads\4.1\MOOD\me\sad"
neutral = r"C:\Users\user\Downloads\4.1\MOOD\me\neutral"
angry = r"C:\Users\user\Downloads\4.1\MOOD\me\angry"

In [4]:
generate_dataset(cascade_path, happy, "happy", n = 200)

In [5]:
generate_dataset(cascade_path, sad, "sad", n = 200)

In [6]:
generate_dataset(cascade_path, neutral, "neutral", n = 200)

In [7]:
generate_dataset(cascade_path, angry, "angry", n = 200)

In [5]:
def prepare_training_data(data_folder):
    faces = []
    labels = []

    emotions = {
        "happy": 0,
        "sad": 1,
        "neutral": 2,
	    "angry": 3
    }

    for emotion in emotions:
        emotion_folder = os.path.join(data_folder, emotion)
        for image_name in os.listdir(emotion_folder):
            image_path = os.path.join(emotion_folder, image_name)
            face = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            face = cv2.resize(face, (200, 200))
            faces.append(face)
            labels.append(emotions[emotion])

    return faces, labels

In [6]:
data_folder = r"C:\Users\user\Downloads\4.1\MOOD\me"  
faces, labels = prepare_training_data(data_folder)

In [7]:
faces = np.array(faces)
labels = to_categorical(labels)  

In [8]:
faces = faces.reshape(-1, 200, 200, 1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(faces, labels, test_size=0.10, random_state=42)

In [10]:
model = Sequential([
    Conv2D(32, (4, 4), activation='relu', input_shape=(200, 200, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')  
])

i:\conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))  

Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 478ms/step - accuracy: 0.3094 - loss: 230.9060 - val_accuracy: 0.6875 - val_loss: 0.7512
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 507ms/step - accuracy: 0.7878 - loss: 0.5377 - val_accuracy: 0.6875 - val_loss: 0.9279
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 491ms/step - accuracy: 0.7191 - loss: 1.2355 - val_accuracy: 0.8375 - val_loss: 0.4084
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 477ms/step - accuracy: 0.8791 - loss: 0.3108 - val_accuracy: 0.9375 - val_loss: 0.2621
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 485ms/step - accuracy: 0.9476 - loss: 0.1599 - val_accuracy: 0.9125 - val_loss: 0.2691
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.9342 - loss: 0.2588 - val_accuracy: 0.8500 - val_loss: 0.4186
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 452ms/step - accuracy: 0.8482 - loss: 0.5808 - val_accuracy: 0.9125 - val_loss: 0.3931
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.8986 - loss: 0.2944 - val_ac

In [16]:
!pip install --user opencv-contrib-python

In [12]:
def face_rec():
  names = ['Unknown', 'neyra']
  model = cv2.face.LBPHFaceRecognizer_create()
  model.read('haarcascade_frontalface_default.xml')
  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        print(label)
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("0"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()

In [13]:
import cv2
from deepface import DeepFace

In [14]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

In [15]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Convert grayscale frame to RGB format
    rgb_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face ROI (Region of Interest)
        face_roi = rgb_frame[y:y + h, x:x + w]

        
        # Perform emotion analysis on the face ROI
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)

        # Determine the dominant emotion
        emotion = result[0]['dominant_emotion']

        # Draw rectangle around face and label with predicted emotion
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('MOOD', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('0'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()
